In [0]:
df = spark.read.table('projeto_data_eng.01_bronze_layer.clientes_raw')

display(df)

In [0]:
from pyspark.sql.functions import col, regexp_replace, count
df = df.withColumn(
    "CPF_MASCARADO",
    regexp_replace(col("CGC_CPF"), r"\d(?=\d)", "X")
)

In [0]:
df = df.drop(
    'CODIGO', 'F_OU_J',"CGC_CPF", 'INSC_RG', 'CLIENTE','CEP','ENDERECO', 'FORNECEDOR', 'PRESTADOR', 
    'VR_LIMITE', 'OBS1', 'BANCO', 'AGENCIA', 'CONTA', 'FONE2', 'FAX', 
    'CELULAR', 'BANCO2', 'AGENCIA2', 'CONTA2', 'TRANSMISSAO', 'ONDE_TRABALHA', 
    'ADMISSAO', 'RENDA_MES', 'CAIXA_POSTAL', 'CLASSE', 'PAI', 'MAE', 'CONJUGE', 
    'QTDE_DEPENDENTES', 'END_TRAB', 'FONE_TRAB', 'NOME_REF_1', 'FONE_REF_1', 
    'NOME_REF_2', 'FONE_REF_2', 'PARENTESCO_FIA', 'CPF_FIA', 'RG_FIA', 'FONE_FIA', 
    'ENDERECO_FIA', 'BAIRRO_FIA', 'CIDADE_FIA', 'CEP_FIA', 'ESTADO_FIA', 
    'PROFISSAO_FIA', 'RENDA_FIA', 'MODIFICADO', 'USUARIO', 'LOJA', 'NUM_FICHA', 
    'DEPENDENTE', 'TITULAR', 'NUM_CONVENIO', 'NAO_AUTORIZADO', 'DT_NASC_FIA', 
    'NUM_BLOQUEIO', 'DATA_BLOQUEIO', 'USU_BLOQUEIO', 'HIST_BLOQUEIO', 
    'COD_MUNICIPIO', 'COD_UF', 'COD_VENDEDOR', 'CLIENTE_PRAMELHOR', 
    'USU_CADASTRO', 'COD_PRAMELHOR', 'CNPJ_CPF_VALIDO', 'INDICACAO', 'PROFISSAO', 
    'SEXO_M_F', 'CODIGO_CLIENTE', 'DATA_DEP_EXCLUIDO', 'TITULAR_DEP_EXCLUIDO', 
    'DT_NASCIMENTO_DEP', 'CODIGO_VALIDADE', 'NOME_VALIDADE', 'DATA_VALIDADE', 
    'USUARIO_VALIDADE', 'OBS_VALIDADE', 'CODIGO_INDICACAO', 'CPF_INDICACAO', 
    'CATEGORIA', 'COMPLEMENTO', 'INSTITUTO_ODC', 'LGPD_USUARIO', 'LGPD_DATA_HORA', 
    'LGPD_CPF', 'LGPD_TELEFONE', 'LGPD_MENSAGEM', 'LGPD_IMAGEM', 'PROTETICO', 
    'PROTETICO_ATIVO', 'PROTETICO_DATA', 'PROTETICO_USUARIO', 'PROTETICO_MOTIVO', 
    'PROTETICO_OBSERVACAO', 'DT_AXON', 'AXON_ID', 'ID_DRCASH', 'DATA_APROVACAO_DRCASH', 
    'VALOR_MAXIMO_DRCASH', 'ID_PAGO_LIVRE', 'NUM_ENDERECO', 'NOME_FIA','cpf_fake'
)
display(df)

In [0]:
df = (
    df.withColumnRenamed('DT_CADASTRO', 'DATA_CADASTRO')
      .withColumnRenamed('DT_ULTMOV', 'DATA_ULTIMO_MOVIMENTO')
      .withColumnRenamed('DT_NASCIMENTO', 'DATA_NASCIMENTO')
      .withColumnRenamed('FONE1', 'FONE')
      .withColumnRenamed('FUNCAO', 'CANAL_AQUISICAO')
      .withColumnRenamed("CPF_MASCARADO", "CPF")
)

In [0]:
df_duplicados = (
    df.groupBy("CPF", "DATA_NASCIMENTO")
      .agg(count("*").alias("qtde"))
      .filter("qtde > 1")
)
display(df_duplicados)

## Filtragem de dados duplicados
## A coluna CPF teve os dados mascarados para não expor informações reais dos clientes, sendo assim, alguns valores ficaram duplicados.

In [0]:
strings_nulos = ['NULL', 'null', ""]
df = df.replace(to_replace=strings_nulos, value=None)

df = df.fillna({
    'EMAIL': 'NAO INFORMADO',
    'CANAL_AQUISICAO': 'NAO INFORMADO'
})

In [0]:
for coluna in df.columns:
    count_nulos = df.filter(df[coluna].isNull()).count()
    print(coluna, count_nulos)

In [0]:
ordem_colunas = ['NOME','CPF','DATA_CADASTRO','CANAL_AQUISICAO','ESTADO','CIDADE','BAIRRO','EMAIL','FONE','DATA_NASCIMENTO','DATA_ULTIMO_MOVIMENTO']

df= df.select(*ordem_colunas)
display(df)

In [0]:
df.write.mode("overwrite").saveAsTable('projeto_data_eng.02_silver_layer.clientes')

In [0]:
df=spark.read.table('projeto_data_eng.02_silver_layer.clientes')
display(df)